In [82]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
import pickle

random_seed = 30011

In [83]:
# read data from iris.csv with column irrigationOn as boolean type
data = pd.read_csv('irrigation.csv')
data.columns = ['soilMoisture', 'temperature', 'humidity', 'irrigationOn']
data

,soilMoisture,temperature,humidity,irrigationOn
0,0,10,20,True
1,0,10,80,True
2,0,25,20,True
3,0,25,80,True
4,200,10,20,True
5,200,10,80,True
6,200,25,20,True
7,200,25,80,True
8,400,10,20,True
9,400,10,80,True


In [84]:
# shuffle data and split into train and test with the ratio 70:30
X_train, X_test, y_train, y_test = train_test_split(data[['soilMoisture', 'temperature', 'humidity']], data['irrigationOn'], test_size = 0.3, random_state = random_seed)
train = {
    'features': X_train.to_numpy(),
    'labels': y_train.to_numpy()
}
test = {
    'features': X_test.to_numpy(),
    'labels': y_test.to_numpy()
}

scaler = StandardScaler()
scaler.fit(train['features'])
train['features-scaled'] = scaler.transform(train['features'])
test['features-scaled'] = scaler.transform(test['features'])

In [85]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 15, 10), random_state=random_seed)
clf.fit(train['features-scaled'], train['labels'])

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(2, 15, 10), random_state=30011,
              solver='lbfgs')

In [86]:
# evaluate the model
predictions = clf.predict(test['features-scaled'])
accuracy = accuracy_score(test['labels'], predictions)
print(f'Accuracy: {accuracy}')

# get predictions for test data and compare with actual label, create a dataframe with an additional column isCorrect and print top 20 rows
df = pd.DataFrame({'actual': test['labels'], 'predicted': predictions})
df['isCorrect'] = df['actual'] == df['predicted']
df.head(10)

Accuracy: 0.8333333333333334


,actual,predicted,isCorrect
0,False,False,True
1,False,False,True
2,False,False,True
3,True,True,True
4,False,True,False
5,False,True,False
6,False,False,True
7,False,False,True
8,False,False,True
9,True,True,True


In [87]:
#save the model
pickle.dump(clf, open('../edge/model.pkl', 'wb'))

In [88]:
model = pickle.load(open('../edge/model.pkl', 'rb'))
predictions = model.predict([[652, 23, 68], [4, 23, 68], [0, 0, 0], [650, 50, 80]])
print(predictions)

[False False  True False]


### Logistic regression

In [90]:
# build a logistic regression model
from sklearn.linear_model import LogisticRegression
logistic_clf = LogisticRegression(random_state=random_seed, max_iter=1000).fit(train['features'], train['labels'])
predictions = logistic_clf.predict(test['features'])
accuracy = accuracy_score(test['labels'], predictions)
print(f'Accuracy: {accuracy}')
predictions = logistic_clf.predict([[652, 23, 68], [4, 23, 68], [0, 0, 0], [650, 50, 80]])
print(predictions)

Accuracy: 0.8333333333333334
[False  True  True False]


In [ ]:
#save the model
pickle.dump(logistic_clf, open('../edge/model-logistic.pkl', 'wb'))